In [33]:
from requests.exceptions import RequestException
from contextlib import closing
import re
from lxml import html
import requests
from datetime import datetime
import pickle

In [34]:
links = set()
with open('sources/cnn_sources.txt') as f:
    for url in f:
        links.add(url)

In [35]:
def date_from_url(url):
    date_search = r"(\d+)\/(\d+)\/(\d+)"
    return datetime(*[int(x) for x in re.search(date_search, url).groups()])

In [36]:
links

{'http://money.cnn.com/2012/11/07/news/economy/climate-change-obama/index.html\n',
 'http://money.cnn.com/2013/04/09/news/climate-change-turbulence/index.html\n',
 'http://money.cnn.com/2013/06/04/news/economy/natural-gas-exports/index.html\n',
 'http://money.cnn.com/2014/09/23/media/dicaprio-un-speech/index.html',
 'http://www.cnn.com/2014/09/21/opinion/sutter-peoples-climate-march/index.html\n',
 'http://www.cnn.com/2014/09/23/opinion/suzman-farmers-adapt-to-climate-change/index.html\n',
 'http://www.cnn.com/2014/11/14/opinion/kohn-climate-change/index.html\n',
 'https://money.cnn.com/2014/09/23/technology/google-climate-change/index.html\n',
 'https://money.cnn.com/2015/10/27/news/climate-change-middle-east/index.html\n',
 'https://money.cnn.com/2015/11/05/news/exxonmobil-climate-change-investigation/index.html\n',
 'https://money.cnn.com/2017/03/29/investing/exxon-trump-paris-climate-change/index.html\n',
 'https://money.cnn.com/2017/04/20/news/corn-farmers-climate-change/index.htm

In [37]:
def get_headline(url, verbose=False):
    # download HTML from simplified page
    base_url = "https://www.textise.net/showText.aspx?strURL="
    if verbose: print("Downloading HTML page...")
    page = requests.get(base_url + url.replace(":", "%253A"))
    tree = html.fromstring(page.content)
    if verbose: print("Finished downloading. Parsing...")
    
    divs = []
    for div in tree.xpath('//div/text()'):
        divs.append(div.rstrip())
    # delete filler text
    delete = ["Español", 'Set edition preference:', u'\xa0 -', "International  Edition", "International Edition", "Find us on", "Read More", 'U.S.', "Arabic", 'International', 'Here are some options:', "Switzerland", " FOLLOW CNN BUSINESS "]
    divs = filter(lambda x: re.sub(' \d+ of \d+', '', x), divs)
    divs = filter(lambda x: x != [], divs)
    divs = filter(lambda x: x != "<div>", divs)
    divs = filter(lambda x: x != "</div>", divs)
    divs = filter(lambda x: x != "\xa0", divs)
    divs = filter(lambda x: x != ".", divs)
    divs = filter(lambda x: x != " |", divs)
    divs = filter(lambda x: x != "", divs)
    divs = filter(lambda x: "/>" not in x, divs)
    divs = filter(lambda x: "</" not in x, divs)
    divs = filter(lambda x: "http" not in x, divs)
    divs = filter(lambda x: "Image:" not in x, divs)
    divs = filter(lambda x: u"\xa0" not in x, divs)
    divs = filter(lambda x: "Facebook Messenger" not in x, divs)
    divs = filter(lambda x: "Hide Caption" not in x, divs)
    divs = filter(lambda x: "MUST WATCH" not in x, divs)
    divs = filter(lambda x: x not in delete, divs)
    #divs = "".join(list(divs)[1:-4])
    #sentences = [x.strip() for x in divs.split(".")][0].split('\n')[0]
    return list(divs)[1]


In [38]:
get_headline("https://www.cnn.com/2014/01/31/world/un-bloomberg/index.html")

'Bloomberg appointed U.N. envoy for cities, climate change'

In [ ]:
date_to_link = {}
for link in links:
    headline = get_headline(link).strip()
    date_to_link[date_from_url(link)] = headline
    print(link, headline)

https://www.cnn.com/2013/09/27/world/climate-change-report/index.html
 Climate change report: It's 'extremely likely' that humans are responsible
https://www.cnn.com/2017/09/14/politics/trump-climate-change-hurricanes/index.html
 Trump dismisses climate change question by contradicting himself on hurricanes
https://www.cnn.com/2017/09/30/politics/resilience-climate-change/index.html
 Trump administration swaps 'climate change' for 'resilience'
https://www.cnn.com/2015/12/11/world/global-climate-change-conference-vote/index.html
 World leaders vote Saturday on global climate change plan
https://www.cnn.com/2013/09/26/world/climate-change-report/index.html
 Climate change: U.N. summary of latest science coming Friday
http://money.cnn.com/2013/06/04/news/economy/natural-gas-exports/index.html
 U.S. steps up natural gas exports
https://www.cnn.com/2013/11/11/opinion/climate-change-hammond/index.html
 Climate change: Why nations, not global talks, are leading the fight
https://www.cnn.com/2

https://www.cnn.com/2015/05/27/politics/ted-cruz-gop-2016-climate-change/index.html
 Texas flooding puts Cruz, GOP in bind on climate change
https://www.cnn.com/2017/04/29/politics/epa-climate-change-website/index.html
 EPA removes climate change information from website
https://www.cnn.com/2012/03/01/us/cnnheroes-nedimyer-coral-reefs/index.html
 Florida group rebuilds vital coral reefs
https://www.cnn.com/2015/09/11/opinions/your-climate-questions-two-degrees/index.html
 Your climate change questions answered
https://www.cnn.com/2016/11/17/africa/china-climate-change-hoax-us-trump/index.html
 China delegate hits back at Trump's climate change hoax claims
http://money.cnn.com/2012/11/07/news/economy/climate-change-obama/index.html
 Climate change is back on the table
https://www.cnn.com/2015/12/11/politics/sean-paul-natasha-bedingfield-toby-gad-climate-change/index.html
 Sean Paul and Natasha Bedingfield get political on climate change
https://www.cnn.com/2016/11/11/politics/climate-ch

https://www.cnn.com/2013/11/03/world/nuclear-energy-climate-change-scientists/index.html
 Climate change warriors: It's time to go nuclear
https://www.cnn.com/2017/06/14/health/climate-change-schools-partner/index.html
 CNN Parents
https://www.cnn.com/2011/10/17/world/asia/shrinking-species-global-warming/index.html
 Climate change is shrinking species, study warns
https://www.cnn.com/2013/04/25/opinion/radford-ruffalo-natural-gas-fracking/index.html
 Don't let America get 'fracked'
https://www.cnn.com/2012/11/01/opinion/field-sandy-climate/index.html
 Climate change is real
https://www.cnn.com/2016/02/25/opinions/sutter-miami-climate-change/index.html
 Packing up because of climate change
https://www.cnn.com/2013/01/23/politics/obama-climate-change/index.html
 Environmentalists want Obama to steer clear of Congress on climate change
https://www.cnn.com/2014/12/30/world/asia/nepal-climate-change-himalayas/index.html
 With climate change, Himalayas' future is warmer, not necessarily bri

In [ ]:
date_to_link

In [ ]:
OUT_FILE = "output_data/output_cnn.pkl"
try:
    with open(OUT_FILE, 'rb') as handle:
        avg_sentiments = pickle.load(handle)
except FileNotFoundError:
    avg_sentiments = {}

In [ ]:
for item in avg_sentiments:
    avg_sentiments[item] = (avg_sentiments[item], date_to_link[item])

In [ ]:
avg_sentiments

In [ ]:
with open("output_data/output_cnn_with_headlines.pkl", 'wb') as f:
    pickle.dump(avg_sentiments, f)

In [27]:
with open("output_data/output_cnn_with_headlines.pkl", 'rb') as f:
    vals = pickle.load(f)
dates = []
headlines = []
sentiments = []

for date in vals:
    dates.append(str(date)[:10])
    headlines.append(vals[date][1].replace(",", ""))
    sentiments.append(vals[date][0])

In [29]:
details = zip(dates, headlines, sentiments)
for x, y, z in sorted(details, key=lambda x: x[0]):
    print("{},{},{}".format(x, y, z))

2011-10-07,Does clean energy kill jobs?,0.04054053993643941
2011-10-17,Climate change is shrinking species study warns,0.09696969773733255
2011-10-21,New climate study deals blow to skeptics,-0.03333333432674408
2011-10-26,'Risk Atlas' assesses climate vulnerability,0.03913043471782104
2011-10-27,The rebirth of renewable energy,0.05135135221722964
2011-11-08,Australia approves carbon emissions tax,-0.047368420582068596
2011-11-29,Kenya's Turkana people facing climate catastrophe,0.030952381768396924
2011-12-03,How climate change has affected your favorite landscapes,0.11666666964689891
2011-12-06,In austere times world needs a climate change 'Plan B',0.12826086985676186
2011-12-09,Why women are world's best climate change defense,0.24255319177470308
2011-12-11,Delegates at climate talks agree to extend efforts of Kyoto Protocol,-0.023529411677051994
2011-12-12,Canada quits Kyoto climate pact,0.0
2011-12-13,Banker: Spending on climate change makes good business sense,0.10689655177552125